# Predição de Sucesso de Startups — Versão aprimorada

Notebook adaptado para maximizar a acurácia dentro das regras do campeonato (somente: Numpy, Pandas, Scikit‑Learn; visualização com Matplotlib/Seaborn).

Principais melhorias:
- Engenharia de features adicionais e transformações (log, rates, flags).
- Tratamento robusto de missing values e outliers (cap por percentis).
- Balanceamento por oversampling apenas no conjunto de treino.
- Seleção de features via RandomForest + SelectFromModel e SelectKBest (mutual_info).
- Busca randômica de hiperparâmetros (RandomizedSearchCV) em RandomForest e HistGradientBoosting.
- Ensemble (VotingClassifier soft) entre estimadores afinados.

Instruções: rode todas as células (o notebook foi pensado para execução em Kaggle/locais com os dados em `../data/`).


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, VotingClassifier
from sklearn.feature_selection import SelectFromModel, SelectKBest, mutual_info_classif
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import time
RANDOM_STATE = 42


## 1) Carregamento dos dados

In [ ]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')
print('Train shape:', train_df.shape)
print('Test shape:', test_df.shape)
display(train_df.head())
display(train_df.info())


## 2) Análise exploratória rápida
- Ver distribuição do target, missing values e correlações iniciais.

In [ ]:
print('Contagem por label:\n', train_df['labels'].value_counts())
print('\nPercentuais:\n', train_df['labels'].value_counts(normalize=True)*100)

missing = train_df.isnull().sum()
missing = missing[missing>0].sort_values(ascending=False)
print('\nColunas com missing (train):')
print(missing)

plt.figure(figsize=(8,5))
sns.countplot(data=train_df, x='labels')
plt.title('Distribuição das labels (train)')
plt.show()


## 3) Engenharia de features
Criamos novas features contínuas e flags que costumam ser informativas para sucesso de startups.

In [ ]:
def create_features(df):
    df = df.copy()
    # Média de idades de funding
    if {'age_first_funding_year','age_last_funding_year'}.issubset(df.columns):
        df['mean_funding_age'] = df[['age_first_funding_year','age_last_funding_year']].mean(axis=1)
    else:
        df['mean_funding_age'] = np.nan
    # duração entre marcos
    if {'age_first_milestone_year','age_last_milestone_year'}.issubset(df.columns):
        df['milestone_duration'] = df['age_last_milestone_year'] - df['age_first_milestone_year']
        df['milestone_duration'] = df['milestone_duration'].fillna(0)
    else:
        df['milestone_duration'] = 0
    # funding por rodada
    if 'funding_total_usd' in df.columns and 'funding_rounds' in df.columns:
        df['funding_per_round'] = df['funding_total_usd'] / df['funding_rounds'].replace(0, np.nan)
        df['funding_per_round'] = df['funding_per_round'].fillna(0)
    else:
        df['funding_per_round'] = 0
    # milestones por rodada
    if 'milestones' in df.columns and 'funding_rounds' in df.columns:
        df['milestones_per_round'] = df['milestones'] / df['funding_rounds'].replace(0, np.nan)
        df['milestones_per_round'] = df['milestones_per_round'].fillna(0)
    else:
        df['milestones_per_round'] = 0
    # flags totais de rounds e localidades
    rounds_flags = [c for c in ['has_VC','has_angel','has_roundA','has_roundB','has_roundC','has_roundD'] if c in df.columns]
    if rounds_flags:
        df['total_round_flags'] = df[rounds_flags].sum(axis=1)
    else:
        df['total_round_flags'] = 0
    loc_flags = [c for c in ['is_CA','is_NY','is_MA','is_TX','is_otherstate'] if c in df.columns]
    if loc_flags:
        df['total_location_flags'] = df[loc_flags].sum(axis=1)
    else:
        df['total_location_flags'] = 0
    # relação por rodada
    if 'relationships' in df.columns and 'funding_rounds' in df.columns:
        df['relationships_per_round'] = df['relationships'] / df['funding_rounds'].replace(0, np.nan)
        df['relationships_per_round'] = df['relationships_per_round'].fillna(0)
    else:
        df['relationships_per_round'] = 0
    # transformações log para features muito assimétricas
    if 'funding_total_usd' in df.columns:
        df['log_funding_total_usd'] = np.log1p(df['funding_total_usd'].fillna(0))
    else:
        df['log_funding_total_usd'] = 0
    # flags simples
    if 'milestones' in df.columns:
        df['has_milestone'] = (df['milestones']>0).astype(int)
    else:
        df['has_milestone'] = 0
    # idade entre first e last funding
    if 'age_first_funding_year' in df.columns and 'age_last_funding_year' in df.columns:
        df['age_between_fundings'] = df['age_last_funding_year'] - df['age_first_funding_year']
        df['age_between_fundings'] = df['age_between_fundings'].fillna(0)
    else:
        df['age_between_fundings'] = 0
    return df

train_df = create_features(train_df)
test_df = create_features(test_df)
display(train_df.head())


## 4) Tratamento de missing values e outliers
- Usamos imputer por mediana nos numéricos. 
- Cap (winsorize) entre 1º e 99º percentil para reduzir influência de outliers.

In [ ]:
# Selecionar colunas numéricas (exceto id e label)
numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [c for c in numeric_cols if c not in ['id','labels']]
numeric_cols[:20]

# Capping por percentis (aplica ao train e test)
def cap_outliers(df, cols, lower_q=0.01, upper_q=0.99):
    df = df.copy()
    for c in cols:
        if c in df.columns:
            low = df[c].quantile(lower_q)
            high = df[c].quantile(upper_q)
            df[c] = df[c].clip(lower=low, upper=high)
    return df

train_df = cap_outliers(train_df, numeric_cols)
test_df = cap_outliers(test_df, numeric_cols)

# Imputação por mediana para variáveis numéricas
imputer = SimpleImputer(strategy='median')
train_df[numeric_cols] = imputer.fit_transform(train_df[numeric_cols])
test_df[numeric_cols] = imputer.transform(test_df[numeric_cols])

print('Missing após imputação (train):')
print(train_df[numeric_cols].isnull().sum().sum())


## 5) Codificação de categóricas
Usamos `get_dummies` com `drop_first=True` para manter compatibilidade com scikit-learn.

In [ ]:
cat_cols = [c for c in train_df.columns if train_df[c].dtype=='object' and c!='id']
cat_cols

train_df = pd.get_dummies(train_df, columns=cat_cols, drop_first=True)
test_df = pd.get_dummies(test_df, columns=cat_cols, drop_first=True)

# Alinhar colunas (adiciona colunas faltantes com 0)
train_cols = set(train_df.columns)
test_cols = set(test_df.columns)
for c in train_cols - test_cols:
    if c!='labels':
        test_df[c] = 0
for c in test_cols - train_cols:
    train_df[c] = 0

# Garantir mesma ordem de colunas
train_df = train_df.reindex(sorted(train_df.columns), axis=1)
test_df = test_df.reindex(sorted(test_df.columns), axis=1)
print('Colunas após encoding:', len(train_df.columns))


## 6) Separação treino/validação e balanceamento (oversampling)
Faremos oversampling **apenas** no conjunto de treino (para evitar vazamento).

In [ ]:
X = train_df.drop('labels', axis=1)
y = train_df['labels']

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE)

print('Train:', X_train.shape, 'Val:', X_val.shape)

# Oversampling simples: replicar minoritária até balancear
def simple_oversample(X, y, random_state=RANDOM_STATE):
    df = pd.concat([X, y], axis=1)
    major = df[df['labels']==0]
    minor = df[df['labels']==1]
    if len(minor)==0:
        return X, y
    ratio = int(len(major)/len(minor))
    if ratio<=1:
        return X, y
    minors_upsampled = minor.sample(n=len(major)-len(minor), replace=True, random_state=random_state)
    df_bal = pd.concat([df, minors_upsampled], axis=0)
    df_bal = df_bal.sample(frac=1, random_state=random_state).reset_index(drop=True)
    return df_bal.drop('labels', axis=1), df_bal['labels']

X_train_bal, y_train_bal = simple_oversample(X_train, y_train)
print('Após oversample (train):', X_train_bal.shape, y_train_bal.value_counts())


## 7) Padronização (StandardScaler) — aplicada apenas nas colunas numéricas
Mantemos as colunas booleanas/categóricas sem escala para preservar interpretações.

In [ ]:
num_cols = [c for c in X_train_bal.columns if X_train_bal[c].dtype in [np.float64, np.int64]]
num_cols = [c for c in num_cols if c!='id']
len(num_cols), num_cols[:20]

scaler = StandardScaler()
X_train_bal_scaled = X_train_bal.copy()
X_val_scaled = X_val.copy()
X_test_scaled = test_df.copy()

X_train_bal_scaled[num_cols] = scaler.fit_transform(X_train_bal[num_cols])
X_val_scaled[num_cols] = scaler.transform(X_val[num_cols])
X_test_scaled[num_cols] = scaler.transform(test_df[num_cols])


## 8) Seleção de features
Usamos duas abordagens e comparamos: SelectKBest(mutual_info) e SelectFromModel (RandomForest importance).
Mantemos a melhor seleção baseada em validação cruzada.

In [ ]:
# 1) SelectKBest (mutual information)
k = min(30, X_train_bal_scaled.shape[1])
skb = SelectKBest(mutual_info_classif, k=k)
skb.fit(X_train_bal_scaled.drop(columns=['id'], errors='ignore'), y_train_bal)
cols_kbest = X_train_bal_scaled.drop(columns=['id'], errors='ignore').columns[skb.get_support()].tolist()
print('Top KBest cols:', len(cols_kbest))

# 2) SelectFromModel (RandomForest)
rf_sel = RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE, n_jobs=-1)
rf_sel.fit(X_train_bal_scaled.drop(columns=['id'], errors='ignore'), y_train_bal)
sfm = SelectFromModel(rf_sel, threshold='median')
sfm.fit(X_train_bal_scaled.drop(columns=['id'], errors='ignore'), y_train_bal)
cols_sfm = X_train_bal_scaled.drop(columns=['id'], errors='ignore').columns[sfm.get_support()].tolist()
print('Cols SelectFromModel:', len(cols_sfm))

# Interseção (mais conservadora)
selected_cols = sorted(list(set(cols_kbest) | set(cols_sfm)))
print('Total selected columns (union):', len(selected_cols))


## 9) Treinamento e tunning (RandomizedSearchCV) — RandomForest e HistGradientBoosting
Buscas são limitadas para execução em tempo razoável; ajuste `n_iter`/`cv` conforme disponibilidade computacional.

In [ ]:
# Preparar datasets com as colunas selecionadas
features = selected_cols
Xtr = X_train_bal_scaled[features]
Xv = X_val_scaled[features]
Xt = X_test_scaled[features]

print('Shapes -> Xtr, Xv, Xt:', Xtr.shape, Xv.shape, Xt.shape)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

rf = RandomForestClassifier(class_weight='balanced', random_state=RANDOM_STATE)
param_dist_rf = {
    'n_estimators': [100,200,300],
    'max_depth': [5,10,20, None],
    'min_samples_split': [2,5,10],
    'min_samples_leaf': [1,2,4]
}
rs_rf = RandomizedSearchCV(rf, param_dist_rf, n_iter=15, cv=cv, scoring='accuracy', n_jobs=-1, random_state=RANDOM_STATE, verbose=1)
t0 = time.time()
rs_rf.fit(Xtr, y_train_bal)
t1 = time.time()
print('RandomForest best params:', rs_rf.best_params_, 'Time (s):', round(t1-t0,1))

# HGB (bom desempenho em dados mistos e rápido)
hgb = HistGradientBoostingClassifier(random_state=RANDOM_STATE)
param_dist_hgb = {
    'max_iter': [100,200,300],
    'max_depth': [3,5,10, None],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'min_samples_leaf': [20,50,100]
}
rs_hgb = RandomizedSearchCV(hgb, param_dist_hgb, n_iter=15, cv=cv, scoring='accuracy', n_jobs=-1, random_state=RANDOM_STATE, verbose=1)
t0 = time.time()
rs_hgb.fit(Xtr, y_train_bal)
t1 = time.time()
print('HGB best params:', rs_hgb.best_params_, 'Time (s):', round(t1-t0,1))


## 10) Ensemble (VotingClassifier) com os melhores estimadores
Usamos `soft` voting (probabilidades) — requer `predict_proba` nos estimadores.

In [ ]:
best_rf = rs_rf.best_estimator_
best_hgb = rs_hgb.best_estimator_
best_lr = LogisticRegression(class_weight='balanced', solver='liblinear', max_iter=200, random_state=RANDOM_STATE)
best_lr.fit(Xtr, y_train_bal)

voting = VotingClassifier(estimators=[('rf', best_rf), ('hgb', best_hgb), ('lr', best_lr)], voting='soft', n_jobs=-1)
t0 = time.time()
voting.fit(Xtr, y_train_bal)
t1 = time.time()
print('Ensemble treinado. Time (s):', round(t1-t0,1))


In [ ]:
# Avaliação no conjunto de validação
yv_pred = voting.predict(Xv)
print('Relatório de classificação (val):')
print(classification_report(y_val, yv_pred))
print('Acurácia (val):', accuracy_score(y_val, yv_pred))
print('Precision:', precision_score(y_val, yv_pred, zero_division=0))
print('Recall:', recall_score(y_val, yv_pred, zero_division=0))
print('F1:', f1_score(y_val, yv_pred, zero_division=0))

cm = confusion_matrix(y_val, yv_pred)
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predito')
plt.ylabel('Verdadeiro')
plt.title('Matriz de confusão (val)')
plt.show()


## 11) Validação cruzada do ensemble
Usamos cross_val_score com cv estratificado para estimar desempenho mais robusto.

In [ ]:
cv_scores = cross_val_score(voting, Xtr, y_train_bal, cv=cv, scoring='accuracy', n_jobs=-1)
print('CV scores:', cv_scores)
print('CV mean accuracy: %.4f +- %.4f' % (cv_scores.mean(), cv_scores.std()))


## 12) Importância das features (RandomForest)
Plota as top-20 features segundo o RandomForest afinado.

In [ ]:
rf_imp = pd.Series(best_rf.feature_importances_, index=features).sort_values(ascending=False)
plt.figure(figsize=(8,6))
sns.barplot(x=rf_imp.values[:20], y=rf_imp.index[:20])
plt.title('Top 20 features (RandomForest)')
plt.tight_layout()
plt.show()


## 13) Treinamento final e submissão
Treinamos o modelo final no conjunto inteiro de treino (com as mesmas transformações aplicadas) e geramos `submission.csv`.

In [ ]:
# Reaplicar todo pipeline no conjunto completo antes de treinar final
X_full = train_df.drop('labels', axis=1)
y_full = train_df['labels']

# Para simplicidade, aplicar oversampling no X_full (opcional) — aqui vamos treinar com a distribuição original e usar class_weight
X_full_scaled = X_full.copy()
X_full_scaled[num_cols] = scaler.transform(X_full[num_cols])

# selecionar colunas
X_full_sel = X_full_scaled[features]

final_model = VotingClassifier(estimators=[('rf', rs_rf.best_estimator_), ('hgb', rs_hgb.best_estimator_), ('lr', best_lr)], voting='soft', n_jobs=-1)
final_model.fit(X_full_sel, y_full)

# Prever no test
X_test_final = X_test_scaled[features]
test_preds = final_model.predict(X_test_final)

submission = pd.DataFrame({'id': test_df['id'], 'labels': test_preds})
submission.to_csv('../data/submission_improved.csv', index=False)
print("Submissão salva em ../data/submission_improved.csv")


## Hipóteses (3) — exemplo que você deve documentar no notebook
1. **Empresas com maior funding por rodada têm maior probabilidade de sucesso** — pois mais capital permite crescimento e resiliência.
2. **Mais milestones por rodada (ou presença de milestones) correlaciona com maior sucesso** — indica execução e progresso.
3. **Presença de investidores (has_VC, has_angel) e múltiplas rodadas (total_round_flags) aumenta chance de sucesso** — sinaliza confiança externa e tração.


## Observações finais
- O notebook aplica técnicas permitidas pelo campeonato (somente bibliotecas autorizadas).
- Ajuste `n_iter` e `cv` nas buscas de hiperparâmetros de acordo com o tempo computacional disponível.
- Execute o notebook no Kaggle/colab/local com os dados em `../data/` e verifique o CSV gerado em `../data/submission_improved.csv`.
